# Imports & load dataset

In [1]:
%load_ext autoreload
%autoreload 2

import zipfile
from airfoil_diffusion.airfoil_datasets import *
from airfoil_diffusion.networks import *
from airfoil_diffusion.trainer import *

In [2]:
if not os.path.exists("./datasets/1_parameter/data/"):
    files=[file for file in os.listdir("./datasets/1_parameter/") if file.endswith(".zip")]
    for file in tqdm(files): 
        f=zipfile.ZipFile("./datasets/1_parameter/"+file,'r')
        for file in f.namelist():
            f.extract(file,"./datasets/1_parameter/data/")
        f.close() 

train_dataset = AirfoilDataset(FileDataFiles("./datasets/1_parameter/train_cases.txt",base_path="./datasets/1_parameter/data/"),
                               data_size=32)

Loading data:  24%|██▍       | 30/125 [00:00<00:00, 296.98it/s]

Loading data: 100%|██████████| 125/125 [00:00<00:00, 294.84it/s]


# Train model

In [7]:
network_configs = {
    "attention_layers": [2, 3],
    "condition_layers": [-2],
    "depth_each_layer": 2,
    "dim_basic": 16,
    "dim_condition": 3,
    "dim_encoded_time": 8,
    "dim_in": 3,
    "dim_multipliers": [1, 2, 4, 4],
    "dim_out": 3,
    "heads_attention": 4,
    "linear_attention": False,
    "skip_connection_scale": 0.707,
    "use_input_condition": True
}

In [8]:
network = AifNet(**network_configs)
network.show_current_configs()

attention_layers: [2, 3]
condition_layers: [-2]
depth_each_layer: 2
dim_basic: 16
dim_condition: 3
dim_encoded_time: 8
dim_in: 6
dim_multipliers: [1, 2, 4, 4]
dim_out: 3
heads_attention: 4
linear_attention: False
skip_connection_scale: 0.707
use_input_condition: True
condition_dim: 0


In [10]:
diffusion_trainer = DiffusionTrainer()
train_configs = {
    "name": "training",
    "save_path": "./training/single_parameter/32/",
    "device": "cuda:0",
    "batch_size_train": 25,
    "shuffle_train": True,
    "num_workers_train": 0,
    "validation_epoch_frequency": 0,
    "optimizer": "AdamW",
    "lr_scheduler": "step",
    "warmup_epoch": 0,
    "record_iteration_loss": False,
    "epochs": 125000,
    "save_epoch": 5000,
    "lr": 0.0001,
    "final_lr": 0.00001
}

In [ ]:
diffusion_trainer.train_from_scratch(network, train_dataset, **train_configs)